# MY FUNCTIONS --------------------------------------------------------

In [1092]:
# return dataframes df1,df2 with only the rows where index in column1 exist also in column2 and vice versa
# and also a third dataframe with the non mergeable rows.

def exclude_differences(df1,df2,column1,column2):
    tmp1 = df1[column1].value_counts().index
    tmp2 = df2[column2].value_counts().index
    my_diff = list(set(tmp1).symmetric_difference(set(tmp2)))
    out_1 = df1.copy()
    out_2 = df2.copy()
    out_3 = out_1.iloc[0:0,:].copy()
    
    for loc in my_diff:
        # we wanna keep track of non meargeable elements
        out_31 = out_1[out_1[column1] == loc] 
        out_32 = out_2[out_2[column2] == loc]
        out_3 = pd.concat([out_3, out_31])
        out_3 = pd.concat([out_3, out_32])
        # now we get rid of these rows
        out_1 = out_1[out_1[column1] != loc] 
        out_2 = out_2[out_2[column2] != loc]
                
    return out_1, out_2, out_3

In [1093]:
def merging_by_name(THE,QS,prob_limit):
    # creating two lists with the names of the universities from the two datasets
    the = THE.copy()
    qs = QS.copy()
    THE_name = list(the.loc[:,'THE_name'])
    QS_name = list(qs.loc[:,'QS_name'])

    #initializing a new column of the THE_df with the corresponding QS name found by the matching function
    # just to control that everything went smoothly
    the['THE_corresponding QS name'] = 'unknown'
    the['prob'] = 'unknown'

    # MATCHING FUNCTION
    # finding the probable corresponding name in the QS dataframe for each university
    for i,THE_uni in enumerate(THE_name):
        QS_uni, prob=process.extractOne(THE_uni, QS_name, scorer=fuzz.token_sort_ratio)
        if prob>=prob_limit: #if prob<87, I observed that the algorithm matches diffeent universities!! 97 is a good limit
            the.loc[(the['THE_name']== THE_uni) , "THE_corresponding QS name"] = QS_uni
            the.loc[(the['THE_name']== THE_uni) , 'prob'] = prob
                     
    # MERGING        
    Unique_df=pd.merge(the,qs, left_on='THE_corresponding QS name', right_on='QS_name', how = 'outer')
    return Unique_df

In [1094]:
def create_unique_df(THE,QS,prob):
    unique_list = []
    for loc in QS['country'].value_counts().sort_index().index:
        the= THE.loc[THE['THE_location'] == loc , :].copy()
        qs  = QS.loc[QS['country'] == loc , :].copy()
        unique = merging_by_name(the,qs,prob)
        unique = unique.dropna(axis=0,how='any')
        unique_list.append(unique)
    unique_out = pd.concat(unique_list)
    return unique_out

In [1095]:
def drop_matched_uni(df,name, list_of_matched_uni):
    out = df.copy()
    for x in out[name]:
        if x in list_of_matched_uni:
            out.drop(out[out[name] == x].index[0], inplace=True)
    return out

In [1046]:
def append_to_non_mergeable(non_mergeable,THE,QS,errors_THE,errors_QS):
    out = non_mergeable.copy()

    for x in errors_THE:
        out = pd.concat([out,THE.loc[THE['THE_name'] == x]])
    for x in errors_QS:
        out = pd.concat([out,QS.loc[QS['QS_name'] == x]])

    return out

#### The next two functions are for text transformation improve fuzz accuracy, we first provide examples of how the libraries work:

fuzz influenced by:  äàö _ 

fuzz not influenced by:  éè'-()

In [1097]:
# example of use for the library unidecode

accented_string = 'Málaga'
# accented_string is of type 'unicode'
unaccented_string = unidecode.unidecode(accented_string)
unaccented_string2 = unidecode.unidecode(string_tmp)
# unaccented_string contains 'Malaga'and is of type 'str'
print(unaccented_string,'\n',unaccented_string2,'\n',string_tmp2 )
print(unidecode.unidecode('äèàäéöû_èüÈ'))
print(unidecode.unidecode(accented_string))


Malaga 
 Ecole Normale Superieure de Lyon 
 Scuola Superiore Sant'Anna Pisa di Studi Universitari e di Perfezionamento
aeaaeou_euE
Malaga


In [1098]:
# example of use for the collections library

table = collections.defaultdict(lambda: None)
table.update({
    ord('é'):'e',
    ord('ô'):'o',
    ord('ö'):'o',
    ord(' '):' ',
    ord('\''):' ',
    ord('-'): ' ',
    ord('\N{NO-BREAK SPACE}'): ' ',
    ord('\N{EN SPACE}'): ' ',
    ord('\N{EM SPACE}'): ' ',
    ord('\N{THREE-PER-EM SPACE}'): ' ',
    ord('\N{FOUR-PER-EM SPACE}'): ' ',
    ord('\N{SIX-PER-EM SPACE}'): ' ',
    ord('\N{FIGURE SPACE}'): ' ',
    ord('\N{PUNCTUATION SPACE}'): ' ',
    ord('\N{THIN SPACE}'): ' ',
    ord('\N{HAIR SPACE}'): ' ',
    ord('\N{ZERO WIDTH SPACE}'): ' ',
    ord('\N{NARROW NO-BREAK SPACE}'): ' ',
    ord('\N{MEDIUM MATHEMATICAL SPACE}'): ' ',
    ord('\N{IDEOGRAPHIC SPACE}'): ' ',
    ord('\N{IDEOGRAPHIC HALF FILL SPACE}'): ' ',
    ord('\N{ZERO WIDTH NO-BREAK SPACE}'): ' ',
    ord('\N{TAG SPACE}'): ' ',
    })
table.update(dict(zip(map(ord,string.ascii_uppercase), string.ascii_lowercase)))
table.update(dict(zip(map(ord,string.ascii_lowercase), string.ascii_lowercase)))
table.update(dict(zip(map(ord,string.digits), string.digits)))

print('123 fôé BAR҉'.translate(table,))
print('Paris-Sorbonne University – Paris 4'.translate(table,))
print(string_tmp2.translate(table,))
print('éöàäèüÒàèò'.translate(table,))


123 foe bar
paris sorbonne university  paris 4
scuola superiore sant anna pisa di studi universitari e di perfezionamento
eo


In [1099]:
def remove_accent_in_name(df,name):
    out=df#.copy()
    for x in out[name]:
        x = unidecode.unidecode(x)
    return out

In [1100]:
def remove_characters_in_name(df,name):
    out=df.copy()
    for x in out[name]:
        x = x.translate(table,)
    return out


# Those where our dataset, from now we try to merge

It appears that there is not the same set of countries in the two ranking. The universities that come from a country not present in the other dataset are thus considered not-mergeable. Note the we replaced "Russian federation " by "Russia" to have a match, assuming that they are the same. 

In [1101]:
# Set russian federation equal russia for comparing countries
THE_df.loc[THE_df['THE_location']=='Russian Federation','THE_location']='Russia'

# find differences in listed countries
tmp1 = QS_df['country'].value_counts().index
tmp2 = THE_df['THE_location'].value_counts().index
my_diff = list(set(tmp1).symmetric_difference(set(tmp2)))
print('Here is my set of country present in only one of the rankings:\n',my_diff)

Here is my set of country present in only one of the rankings:
 ['Malaysia', 'Argentina', 'Chile', 'Brazil', 'India', 'Luxembourg', 'Israel', 'Saudi Arabia', 'Mexico']


## STEP 1 remove unis of countries that are not in both rankings

In [1104]:
QS_step1, THE_step1, non_mergeable = exclude_differences(QS_df,THE_df,'country','THE_location')

print('QS_df remaining rows: ',QS_step1.shape[0],'    THE_df remaining rows: ',THE_step1.shape[0])
print('Rows in non-mergeable: ',non_mergeable.shape[0])
QS_step1.sort_values('country',inplace=True)
THE_step1.sort_values('THE_location',inplace=True)

QS_df remaining rows:  187     THE_df remaining rows:  199
Rows in non-mergeable:  14


##### Note that non_meargeable contains only a subset of all the non-meargeables rows, we will append rows to it as the analysis continues

## STEP 2 take out 100% fitting unis names

In [1116]:
Unique_df = create_unique_df(THE_step1,QS_step1,100)

# list of matched unis by ranking
QS_matched_uni = list(Unique_df['QS_name'])
THE_matched_uni = list(Unique_df['THE_name'])

# update the remaining ranking dataframes
THE_step2 = drop_matched_uni(THE_step1,'THE_name',THE_matched_uni)
QS_step2 = drop_matched_uni(QS_step1,'QS_name', QS_matched_uni)

# print to check that the dataframes have the correct size:
print('merged rows at this step: ',Unique_df.shape[0])
print('Non mergeable dataframe #rows: ',non_mergeable.shape[0])
print('THE before: ',THE_step1.shape[0],'   THE after: ',THE_step2.shape[0])
print('QS before: ',QS_step1.shape[0],'   QS after: ',QS_step2.shape[0])

merged rows at this step:  107
Non mergeable dataframe #rows:  16
THE before:  199    THE after:  92
QS before:  187    QS after:  80


## STEP 3 go to 90%

In [1115]:
THE_step2_tmp = remove_accent_in_name(THE_step2,'THE_name') # try to remove accents
QS_step2_tmp = remove_accent_in_name(QS_step2,'QS_name') # but without succes

Unique_df2 = create_unique_df(THE_step2_tmp,QS_step2_tmp,90) #90 since 100 gave nothing

# list of matched unis by ranking (at this step)
THE_matched_uni = list(Unique_df2['THE_name'])
QS_matched_uni = list(Unique_df2['QS_name'])

# update the remaining ranking dataframes
THE_step3 = drop_matched_uni(THE_step2,'THE_name',THE_matched_uni)
QS_step3 = drop_matched_uni(QS_step2,'QS_name', QS_matched_uni)

# usual cross check
print('merged rows at this step: ',Unique_df2.shape[0])
print('Non mergeable dataframe #rows: ',non_mergeable.shape[0])
print('THE before: ',THE_step2.shape[0],'   THE after: ',THE_step3.shape[0])
print('QS before: ',QS_step2.shape[0],'   QS after: ',QS_step3.shape[0])

merged rows at this step:  37
Non mergeable dataframe #rows:  16
THE before:  92    THE after:  55
QS before:  80    QS after:  43


## STEP 4 go to 80%

In [1114]:
THE_step3_tmp = remove_characters_in_name(THE_step3,'THE_name') # tried to remove special characters
QS_step3_tmp = remove_characters_in_name(QS_step3,'QS_name') # doesn't work though

Unique_df3 = create_unique_df(THE_step3_tmp,QS_step3_tmp,80) 

# errors made
errors = ["Trinity College Dublin",'University College Dublin']

# update the non-mergeable dataframe
non_mergeable = pd.concat([non_mergeable,THE_step3.loc[THE_step3['THE_name'] == "Trinity College Dublin"]]) 
non_mergeable = pd.concat([non_mergeable,QS_step3.loc[QS_step3['QS_name'] == "University College Dublin"]]) 

Unique_df3 = Unique_df3.loc[Unique_df3['THE_name'] != "Trinity College Dublin"] #the only one with an error

# list of matched unis by ranking (at this step)
THE_matched_uni = list(Unique_df3['THE_name'])  + errors
QS_matched_uni = list(Unique_df3['QS_name']) + errors

# update the remaining ranking dataframes
THE_step4 = drop_matched_uni(THE_step3,'THE_name',THE_matched_uni)
QS_step4 = drop_matched_uni(QS_step3,'QS_name', QS_matched_uni)

# remove duplicates ()usefull mainly if we runs multiples times this cell
non_mergeable = non_mergeable.drop_duplicates()

# usual cross check 
print('merged rows at this step: ',Unique_df3.shape[0])
print('Non mergeable dataframe #rows: ',non_mergeable.shape[0])
print('THE before: ',THE_step3.shape[0],'   THE after: ',THE_step4.shape[0])
print('QS before: ',QS_step3.shape[0],'   QS after: ',QS_step4.shape[0])

merged rows at this step:  8
Non mergeable dataframe #rows:  16
THE before:  55    THE after:  46
QS before:  43    QS after:  34


## STEP 5 go to 70%

In [1118]:
# df created with the matching function, but some errors are present
Unique_df4 = create_unique_df(THE_step4,QS_step4,70) 

# errors at this step
errors = ['University of Sussex','University of Reading',\
          'University of East Anglia','University of Bath',\
          'University of Leicester','University of Dundee']

# errors by ranking
errors_THE = ['University of Sussex','University of East Anglia','University of Dundee','University of Leicester']
errors_QS = ['University of Reading','University of Bath']

# clean the unique_df from its errors
for x in Unique_df4['QS_name']:
    if x in errors:
        Unique_df4 = Unique_df4[Unique_df4['QS_name'] != x]
for x in Unique_df4['THE_name']:
    if x in errors:
        Unique_df4 = Unique_df4[Unique_df4['THE_name'] != x]

#update non mergeable df
non_mergeable = append_to_non_mergeable(non_mergeable,THE_step4,QS_step4,\
                                                    errors_THE,errors_QS)
#drop duplicates
non_mergeable = non_mergeable.drop_duplicates()

# list of matched unis by ranking (at this step)
THE_matched_uni = list(Unique_df4['THE_name'])  + errors_THE
QS_matched_uni = list(Unique_df4['QS_name']) + errors_QS

# update the remaining ranking dataframes
THE_step5 = drop_matched_uni(THE_step4,'THE_name',THE_matched_uni)
QS_step5 = drop_matched_uni(QS_step4,'QS_name', QS_matched_uni)

# usual cross check 
print('merged rows at this step: ',Unique_df4.shape[0])
print('Non mergeable dataframe #rows: ',non_mergeable.shape[0])
print('THE before: ',THE_step4.shape[0],'   THE after: ',THE_step5.shape[0])
print('QS before: ',QS_step4.shape[0],'   QS after: ',QS_step5.shape[0])

merged rows at this step:  3
Non mergeable dataframe #rows:  22
THE before:  46    THE after:  39
QS before:  34    QS after:  29


## Final STEP all.

In [1119]:
# create merged dataframe with matching function. But some errors are present
Unique_df5 = create_unique_df(THE_step5,QS_step5,10)

# correct matchings for THE_ranking (since they are unique due to our merging function)
rights = ['University of Tübingen','LMU Munich','University of Freiburg','Scuola Superiore Sant’Anna']

# errors made: (incorrect matching)
errors = list(set(rights).symmetric_difference(set(list(Unique_df5['THE_name']))))

for x in errors:
    Unique_df5 = Unique_df5[Unique_df5['THE_name'] != x] 


# list of matched unis by ranking (at this step)
THE_matched_uni = list(Unique_df5['THE_name']) 
QS_matched_uni = list(Unique_df5['QS_name']) 

# update the remaining ranking dataframes
THE_step6 = drop_matched_uni(THE_step5,'THE_name',THE_matched_uni)
QS_step6 = drop_matched_uni(QS_step5,'QS_name', QS_matched_uni)

# update non_mergeables unis dataframe
non_mergeable = pd.concat([non_mergeable,THE_step6,QS_step6])
non_mergeable = non_mergeable.drop_duplicates()

# usual cross check 
print('merged rows at this step: ',Unique_df5.shape[0])
print('Non mergeable dataframe #rows: ',non_mergeable.shape[0])
print('THE before: ',THE_step5.shape[0],'   THE after: ',THE_step6.shape[0])
print('QS before: ',QS_step5.shape[0],'   QS after: ',QS_step6.shape[0])

merged rows at this step:  4
Non mergeable dataframe #rows:  82
THE before:  39    THE after:  35
QS before:  29    QS after:  25


## Final check: 

In [1122]:
merged_df = pd.concat([Unique_df,Unique_df2,Unique_df3,Unique_df4,Unique_df5])
num_merged = merged_df.shape[0]

non_mergeable = non_mergeable.drop_duplicates()
num_non_merged = non_mergeable.shape[0]
print ('merged: ',num_merged,'  non-merged',num_non_merged,'\nmissing',(200-num_merged)*2-num_non_merged)
print('\nif 0 missing, it is probable the merge is successfull')

merged:  159   non-merged 82 
missing 0

if 0 missing, it is probable the merge is successfull
